# Validation Croisée et Analyse de l'Underfitting/Overfitting

Dans ce notebook, nous allons explorer :
- Les courbes d'apprentissage pour détecter l'underfitting/overfitting
- La validation croisée avec KFold
- L'optimisation des hyperparamètres avec GridSearchCV

In [ ]:
# Imports nécessaires
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import (
    train_test_split,
    KFold,
    cross_val_score,
    learning_curve,
    GridSearchCV
)
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Chargement des Données

In [ ]:
# Chargement du dataset
data = fetch_california_housing(as_frame=True)
df = data.frame.copy()

X = df.drop("MedHouseVal", axis=1)
y = df["MedHouseVal"]

## 2. Analyse de l'Underfitting/Overfitting

In [ ]:
def plot_learning_curve(train_sizes, train_scores, val_scores):
    """
    Fonction pour tracer la courbe d'apprentissage
    """
    train_mean = np.mean(train_scores, axis=1)
    val_mean = np.mean(val_scores, axis=1)

    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, 'o-', label='Score d\'entraînement')
    plt.plot(train_sizes, val_mean, 'o-', label='Score de validation')
    plt.xlabel("Nombre d'exemples d'entraînement")
    plt.ylabel("Score")
    plt.legend()
    plt.grid()
    plt.title("Courbe d'Apprentissage")
    plt.show()

In [ ]:
# Génération de la courbe d'apprentissage avec Ridge
model = Ridge(alpha=1.0)
train_sizes, train_scores, val_scores = learning_curve(
    model, X, y, 
    cv=5, 
    scoring="r2",
    train_sizes=np.linspace(0.1, 1.0, 5),
    n_jobs=-1
)

plot_learning_curve(train_sizes, train_scores, val_scores)

## 3. Validation Croisée avec KFold

In [ ]:
# Configuration de la validation croisée
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Évaluation avec régression linéaire
model = LinearRegression()
cv_scores = cross_val_score(model, X, y, cv=kf, scoring="r2")

print("Scores de validation croisée (R²):", cv_scores)
print("Score moyen:", np.mean(cv_scores))
print("Écart-type:", np.std(cv_scores))

## 4. Optimisation des Hyperparamètres avec GridSearchCV

In [ ]:
# Définition de la grille de paramètres pour RandomForest
param_grid = {
    "n_estimators": [10, 50, 100],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5]
}

# Configuration et exécution de la recherche sur grille
grid_search = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

grid_search.fit(X, y)

In [ ]:
# Affichage des résultats
print("Meilleurs paramètres trouvés:", grid_search.best_params_)
print("Meilleur RMSE:", -grid_search.best_score_)

# Affichage des résultats détaillés
results = pd.DataFrame(grid_search.cv_results_)
results = results.sort_values('rank_test_score')
print("\nTop 5 des combinaisons de paramètres:")
print(results[['params', 'mean_test_score', 'std_test_score']].head())